In [58]:
import yfinance as yf
import os
import pandas as pd
from datetime import datetime, date, timedelta
from dateutil.parser import parse as dtparse

# ------------------ USER SETTINGS ------------------
MAX_EXPIRIES_PER_TICKER = 8     # to avoid rate-limit pain (nearest 7 + Jan'26 if present)
LAST_PRICE_MAX = 1.00           # <= $2.00 contracts only
VOL_MIN = 300                   # minimum volume to be considered
VOL_OI_MIN = 3.0                # volume/open interest threshold
END_DATE_CUTOFF = "2026-01-31"  # scan expiries up to end-Jan 2026

base_date = datetime.now().strftime('%Y-%m-%d')
prefix = f"unusual_options_scan_{base_date}"
ext = ".csv"

# Find the highest existing index for today
existing_indices = []
for fname in os.listdir('.'):
    if fname.startswith(prefix) and fname.endswith(ext):
        # Extract the number between last underscore and .csv
        try:
            num = int(fname[len(prefix)+1:-len(ext)])
            existing_indices.append(num)
        except ValueError:
            pass

# Determine next index
next_index = max(existing_indices) + 1 if existing_indices else 1
SAVE_CSV = f"{prefix}_{next_index}{ext}"

print(f"Next file to save: {SAVE_CSV}")

# SAVE_CSV = f"unusual_options_scan_{datetime.now().strftime('%Y-%m-%d')}.csv"
# ---------------------------------------------------

# NASDAQ-100 tickers (quickly updated set; harmless if a few changed)
NASDAQ100 = [
    "AAPL","MSFT","NVDA","AMZN","META","GOOGL","GOOG","TSLA","AVGO","COST",
    "NFLX","PEP","ADBE","AMD","LIN","TMUS","CSCO","QCOM","TXN","AMAT",
    "INTU","HON","INTC","BKNG","SBUX","MU","AMGN","PDD","REGN","LRCX",
    "ADP","ISRG","ABNB","MDLZ","VRTX","ASML","GILD","ADI","PANW","KLAC",
    "PYPL","CRWD","CSX","WDAY","CHTR","MAR","NXPI","ROP","AEP","KDP",
    "MELI","FTNT","ORLY","SNPS","CDNS","MNST","CTAS","DXCM","PCAR","LULU",
    "MRVL","MCHP","ROST","EXC","ODFL","ADSK","ATVI","IDXX","EA",
    "PAYX","CTSH","TEAM","XEL","WDAY","DDOG","ZS","SPLK","BKR","ALGN",
    "AZN","CEG","VRSK","SIRI","PDD","LCID","RIVN","BIDU","JD","BMRN",
    "DOCU","VRSN","NTES","MRNA","ANSS","CSGP","CHKP","MTCH","CRWD","OKTA",
    "NEE", "JNJ", "SMCI", "STZ", "TMQ", "PLTR", "XYZ", "HOOD", "ORCL", "UPST",
    "TSM", "SHOP", "SPOT", "LLY", "HIMS", "UNH", "DELL", "COIN", "OSCR", "SNOW",
    "QUBT", "RGTI", "CRWV", "RKLB", "BA", "QCOM", "PANW", "JPM", "GS", "BABA", "BIDU", "USAR", "ONON", "VIX"
]
# Deduplicate (list may contain a couple repeats above)
TICKERS = sorted(list(dict.fromkeys(NASDAQ100)))

def safe_option_chain(tkr, exp):
    """Return (calls, puts) DataFrames or (None, None) on failure."""
    try:
        oc = tkr.option_chain(exp)
        c = oc.calls.copy()
        p = oc.puts.copy()
        c["type"] = "CALL"
        p["type"] = "PUT"
        for df in (c, p):
            df["expiration"] = exp
        return c, p
    except Exception:
        return None, None

def pick_expiries(all_exps):
    """
    Choose a practical subset:
    - nearest expiries in order, up to MAX_EXPIRIES_PER_TICKER - 1
    - plus Jan 2026 (3rd Friday or any Jan-2026 date in list) if present
    - only expiries <= END_DATE_CUTOFF and >= today
    """
    today = date.today().isoformat()
    OFFSET_DAYS = 2  # for example, start looking 7 days from today
    today_offset = (date.today() + timedelta(days=OFFSET_DAYS)).isoformat()
    cutoff = END_DATE_CUTOFF
    exps = [e for e in all_exps if today_offset <= e <= cutoff]
    exps_sorted = sorted(exps)
    chosen = exps_sorted[:max(0, MAX_EXPIRIES_PER_TICKER - 1)]
    # try to include a Jan 2026 expiry if available
    jan26 = [e for e in exps_sorted if e.startswith("2026-01")]
    if jan26:
        jan_pick = jan26[0]
        if jan_pick not in chosen:
            chosen.append(jan_pick)
    return chosen

def scan_ticker(ticker):
    tkr = yf.Ticker(ticker)
    try:
        all_exps = tkr.options
    except Exception:
        return pd.DataFrame()

    if not all_exps:
        return pd.DataFrame()

    exps = pick_expiries(all_exps)
    rows = []
    for exp in exps:
        calls, puts = safe_option_chain(tkr, exp)
        if calls is None:
            continue
        df = pd.concat([calls, puts], ignore_index=True)

        # Clean columns (Yahoo schema can vary slightly)
        for col in ["lastPrice","volume","openInterest","strike"]:
            if col not in df.columns:
                df[col] = 0

        # Filter rules:
        # 1) lastPrice <= $2.00
        # 2) decent volume
        # 3) unusual-ish vol/oi ratio
        df["vol_oi"] = df["volume"] / df["openInterest"].replace(0, 1)
        flt = (
            (df["lastPrice"] <= LAST_PRICE_MAX) &
            (df["volume"] >= VOL_MIN) &
            (df["vol_oi"] >= VOL_OI_MIN)
        )
        df = df.loc[flt, ["contractSymbol","type","strike","lastPrice","volume","openInterest","vol_oi","expiration"]]
        df["ticker"] = ticker

        # Add a simple score to rank results (volume * vol/oi)
        df["score"] = df["volume"] * df["vol_oi"]
        rows.append(df)

    if not rows:
        return pd.DataFrame()
    out = pd.concat(rows, ignore_index=True)
    return out

def main():
    all_hits = []
    for i, tk in enumerate(TICKERS, 1):
        print(f"[{i:3d}/{len(TICKERS)}] Scanning {tk} ...")
        hits = scan_ticker(tk)
        if not hits.empty:
            all_hits.append(hits)

    if not all_hits:
        print("No matches found with current filters. Consider lowering VOL_MIN or VOL_OI_MIN.")
        return

    df = pd.concat(all_hits, ignore_index=True)

    # Find the top 10 *stocks* with strongest signals:
    # Rank per-ticker by max score, keep top 10 tickers, then show their top rows
    per_ticker = df.groupby("ticker")["score"].max().reset_index().sort_values("score", ascending=False)
    top10_tickers = per_ticker["ticker"].head(10).tolist()
    final = (df[df["ticker"].isin(top10_tickers)]
             .sort_values(["ticker","score"], ascending=[True, False]))

    # Save CSV
    final_cols = ["ticker","type","strike","lastPrice","volume","openInterest","vol_oi","expiration","contractSymbol","score"]
    final[final_cols].to_csv(SAVE_CSV, index=False)
    print(f"\nSaved {len(final)} matches to: {SAVE_CSV}")

    # Print a quick summary to console
    print("\n=== Top 10 tickers (by max score) ===")
    for _, row in per_ticker[per_ticker["ticker"].isin(top10_tickers)].iterrows():
        print(f"{row['ticker']:>5}  score={row['score']:.1f}")

    print("\n=== Sample rows ===")
    print(final[final_cols].head(25).to_string(index=False))

if __name__ == "__main__":
    main()


Next file to save: unusual_options_scan_2025-10-12_1.csv
[  1/127] Scanning AAPL ...
[  2/127] Scanning ABNB ...
[  3/127] Scanning ADBE ...
[  4/127] Scanning ADI ...
[  5/127] Scanning ADP ...
[  6/127] Scanning ADSK ...
[  7/127] Scanning AEP ...
[  8/127] Scanning ALGN ...
[  9/127] Scanning AMAT ...
[ 10/127] Scanning AMD ...
[ 11/127] Scanning AMGN ...
[ 12/127] Scanning AMZN ...
[ 13/127] Scanning ANSS ...
[ 14/127] Scanning ASML ...
[ 15/127] Scanning ATVI ...
[ 16/127] Scanning AVGO ...
[ 17/127] Scanning AZN ...
[ 18/127] Scanning BA ...
[ 19/127] Scanning BABA ...
[ 20/127] Scanning BIDU ...
[ 21/127] Scanning BKNG ...
[ 22/127] Scanning BKR ...
[ 23/127] Scanning BMRN ...
[ 24/127] Scanning CDNS ...
[ 25/127] Scanning CEG ...
[ 26/127] Scanning CHKP ...
[ 27/127] Scanning CHTR ...
[ 28/127] Scanning COIN ...
[ 29/127] Scanning COST ...
[ 30/127] Scanning CRWD ...
[ 31/127] Scanning CRWV ...
[ 32/127] Scanning CSCO ...
[ 33/127] Scanning CSGP ...
[ 34/127] Scanning CSX ...
[